## Original Publication

See the original publication of this work in the following link: https://www.swsc-journal.org/articles/swsc/abs/2021/01/swsc210024/swsc210024.html

Aminalragia-Giamini, Sigiava, et al. "Solar Energetic Particle Event occurrence prediction using Solar Flare Soft X-ray measurements and Machine Learning." Journal of Space Weather and Space Climate 11 (2021): 59.


# Loading Data 

## Importing libraries and data files

In [1]:
import os
import tensorflow as tf
import datetime
import sys
from os import path
import numpy as np
import pandas as pd
from tensorflow import keras 
import xgboost as xgb

x_train = pd.read_pickle("x_train.pkl")  
x_test = pd.read_pickle("x_test.pkl")  
y_train = pd.read_pickle("y_train.pkl")  
y_test = pd.read_pickle("y_test.pkl")  

# number of different classes
n_SEPS =  226
n_flares = 17875
n_sample = n_SEPS +n_flares
columns_to_use = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','17','20','21','22','28','39','40',
                       '41','42','43','44','45','46','47','48']
number_of_classes = 2 

2023-08-23 10:56:21.801644: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
x_total = x_train.append(x_test)
y_total = y_train.append(y_test)

/var/folders/vs/s8nbsg4x7nd44x9k_00v8f0m0000gq/T/ipykernel_30406/4287163944.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x_total = x_train.append(x_test)
/var/folders/vs/s8nbsg4x7nd44x9k_00v8f0m0000gq/T/ipykernel_30406/4287163944.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y_total = y_train.append(y_test)


## Prepare validation

Establish validation method below

In [5]:
from sklearn.model_selection import LeaveOneOut
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import cv

sample_weight = compute_sample_weight(class_weight='balanced',y=y_total)
d_train = xgb.DMatrix(x_total, label=y_total)

params={"objective" : "binary:logistic", 'learning_rate' :0.1, 'max_depth':10, 'scale_pos_weight':1}

xgb_cv = cv(dtrain=d_train, params =params, nfold=50, metrics ="auc", num_boost_round=50,as_pandas =True)

bst=xgb.XGBClassifier(max_depth=10,learning_rate=0.1, n_estimators=100, objective='binary:logistic', scale_pos_weight=1,eval_metric='logloss')


[10:58:40] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:58:41] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:58:41] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:58:41] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:58:41] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_b2k4a2hea3/croot/xgboost-split_1675457783214/work/src/learner.cc:767: 
Parameters: { "n_estimators" } are not used.

[10:58:41] WARNING: /var/folders/sy/f16zz6x50xz311

In [11]:
xgb_cv

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.893955,0.008803,0.865037,0.108233
1,0.915679,0.017160,0.879621,0.105628
2,0.922297,0.014869,0.884749,0.103581
3,0.928629,0.012565,0.891464,0.101359
4,0.930319,0.010802,0.899493,0.091282
5,0.931055,0.010517,0.907275,0.090747
6,0.933298,0.010700,0.909254,0.089689
7,0.940896,0.009004,0.909487,0.090356
8,0.943678,0.009868,0.911373,0.089155
9,0.946932,0.009272,0.922521,0.083234


## Example of Training proccedure


Let's now train our model

In [8]:
bst=xgb.XGBClassifier(max_depth=10,learning_rate=0.01, n_estimators=50, objective='binary:logistic')
# fit model
model_used=bst.fit(x_total, y_total)

# Visualziating results (simple metrics)

Let's see the results

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

predictions = model_used.predict(x_test)
predictions_binary = np.argmax(predictions, axis = 1)
y_test_binary = np.argmax(y_test, axis=1)
    
cr = classification_report(y_test_binary, predictions_binary)
cm = confusion_matrix(y_test_binary, predictions_binary)

print(cm)
print("Class 1 accuracy")
print(cm[0,0]/(cm[0,0]+cm[0,1]))

print("Class 2 accuracy")
print(cm[1,1]/(cm[1,0]+cm[1,1]))

#print(model_used.summary())
print(cr)

AxisError: axis 1 is out of bounds for array of dimension 1

# Visualziating results (Plots)

Let's see the results

In [13]:
bst.feature_importances_

array([0.00814376, 0.01327463, 0.01484632, 0.00787467, 0.02193077,
       0.01172764, 0.        , 0.        , 0.        , 0.        ,
       0.01498673, 0.00882444, 0.10886821, 0.03972897, 0.01323617,
       0.00666374, 0.01276817, 0.01295464, 0.01944197, 0.00858052,
       0.        , 0.        , 0.01888355, 0.01548385, 0.01333147,
       0.00981255, 0.34151903, 0.06203757, 0.01205044, 0.01337198,
       0.01716927, 0.01633044, 0.00681656, 0.01264735, 0.01296095,
       0.01367681, 0.        , 0.        , 0.        , 0.01914397,
       0.00777842, 0.00943802, 0.00512267, 0.01109561, 0.00663682,
       0.01324036, 0.00990741, 0.01108752, 0.01660605], dtype=float32)

In [25]:
import matplotlib.pyplot as plt

val_f1 = model_used.history['val_f1_m']
val_loss= model_used.history['val_loss']
val_acc= model_used.history['val_acc']
val_prec= model_used.history['val_precision_m']

train_f1 = model_used.history['f1_m']
train_loss = model_used.history['loss']
train_acc = model_used.history['acc']
train_prec = model_used.history['precision_m']

plt.figure(figsize=(11.69,8.27))
plt.subplot(2, 2, 1)
plt.plot(train_f1)
plt.plot(val_f1)
plt.ylabel('f1 score',fontsize=22)
plt.legend(['train', 'test'], loc='lower right',fontsize=16)

plt.subplot(2, 2, 2)
plt.plot(train_loss)
plt.plot(val_loss)
plt.ylabel('loss',fontsize=22)

plt.subplot(2, 2, 3)
plt.plot(train_acc)
plt.plot(val_acc)
plt.xlabel('epochs',fontsize=22)
plt.ylabel('acc',fontsize=22)

plt.subplot(2, 2, 4)
plt.plot(train_prec)
plt.plot(val_prec)
plt.xlabel('epochs',fontsize=22)
plt.ylabel('precision',fontsize=22)

plt.show()



AttributeError: 'XGBClassifier' object has no attribute 'history'